
1. **pdf2image** -> a Python library that allows you to convert PDF files into images. 
2. **convert_from_path** -> a function specifically converts PDF files located at a given file path into a list of PIL (Python Imaging Library) images. This function simplifies the process of extracting images from PDF documents for further processing or analysis in Python programs.

In [53]:
from pdf2image import convert_from_path

If you're running the code on Windows, you'll need to specify the path to __Poppler__ .
Example -  poppler_path = r'C:\poppler-21.02.0\Library\bin'

In [54]:
pages = convert_from_path(r'/Users/sohamjana/Downloads/source-code/4_project_medical_data_extraction/backend/notebooks/docs/prescription/pre_1.pdf')

Here, **pages** is an array of PIL Images.
Pillow/PIL: Python libraries for opening, editing, and saving images, with Pillow being the actively maintained version.


Here, the length of **pages** is 1, as there is a single picture in this pdf.

In [55]:
len(pages)

1

In [56]:
pages[0].show()


**pytesseract** imports Tesseract OCR (Optical Character Recognition) engine for text extraction from images in Python.

In [57]:
import pytesseract


In [58]:
from pytesseract import image_to_string

This line of code sets the path to the Tesseract executable file (tesseract) for the pytesseract library. 


In [59]:
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'

In [60]:
text = pytesseract.image_to_string(pages[0], lang='eng')
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Maria Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

—momennannenncmneneunnmnnnnninsissiyoinnitnahaadaanih issn earnttneenrenen:

Prednisone 20 mg
Lialda 2.4 gram

3 days,

or 1 month



The shadowed part of the picture doesn't produce a perfect output. We'll use OpenCV to process the image and then convert it into a string.

OpenCV (Open Source Computer Vision Library) is an open-source library of programming functions mainly aimed at real-time computer vision tasks. It provides tools and algorithms for image and video processing, including object detection, facial recognition, and feature extraction, among others.


In OpenCV, 0 typically represents black and 255 represents white, adhering to the standard grayscale pixel intensity range.


Simple thresholding applies a fixed threshold value to the entire image, while adaptive thresholding computes thresholds for small image regions independently, resulting in more adaptive and localized thresholding.

In [71]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        63,
        11
    )
    return processed_image

Sure, let's break down the code line by line:

1. `import numpy as np`: Imports the NumPy library under the alias `np`. NumPy is used for numerical computations in Python.

2. `import cv2`: Imports the OpenCV library. OpenCV is a popular library used for computer vision tasks, such as image processing and object detection.

3. `from PIL import Image`: Imports the `Image` class from the Python Imaging Library (PIL). PIL is a library used for image manipulation tasks.

4. `def preprocess_image(img):`: Defines a function named `preprocess_image` that takes one parameter `img`.

5. `gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)`: Converts the input image to grayscale using OpenCV's `cvtColor` function. It first converts the image to a NumPy array and then applies the conversion to grayscale.

6. `resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)`: Resizes the grayscale image to 1.5 times its original size using OpenCV's `resize` function. The interpolation method used is linear interpolation.

7. `processed_image = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 63, 11)`: Applies adaptive thresholding to the resized image using OpenCV's `adaptiveThreshold` function. This method adapts the threshold value for each pixel based on the local neighborhood of the pixel. Here, it uses a Gaussian window to calculate the threshold and sets pixels above the threshold to 255 (white) and below to 0 (black).

8. `return processed_image`: Returns the processed image from the function.

In [75]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [76]:
pytesseract.pytesseract.tesseract_cmd=r'/opt/homebrew/bin/tesseract'
text = pytesseract.image_to_string(img, lang='eng')
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Marta Sharapova Date: 2/11/2022

Address: 9 tennis court, new Russia, DC

K

Prednisone 20 mg
Lialda 2.4 gram

Directions:

Prednisone, Taper 5 mg every 3 days,
Finish in 2.5 weeks a
Lialda - take 2 pill everyday for 1 month

Refill: 2 times



In [74]:
text = '''
Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222
Name: Marta Sharapova Date: 5/11/2022
Address: 9 tennis court, new Russia, DC

 
 

Prednisone 20 mg
Lialda 2.4 gram
Directions:
Prednisone, Taper 5 mg every 3 days,
Finish in 2.5 weeks -
Lialda - take 2 pill everyday for 1 month
Refill: 3 times
'''

In [51]:
import re

pattern = 'Name:(.*)Date'

matches = re.findall(pattern, text)
matches[0].strip()

'Marta Sharapova'

In [42]:
pattern = 'Address:(.*)\n'

matches = re.findall(pattern, text)
matches[0].strip()

'9 tennis court, new Russia, DC'

In [46]:
pattern = 'Address[^\n]*(.*)Directions'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Prednisone 20 mg\nLialda 2.4 gram'

In [45]:
pattern = 'Directions:(.*)Refill'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Prednisone, Taper 5 mg every 3 days,\nFinish in 2.5 weeks -\nLialda - take 2 pill everyday for 1 month'

In [47]:
pattern = 'Refill:(.*)times'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'3'